In [1]:
%pylab inline
from scipy import ndimage
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid.anchored_artists import AnchoredText
import sys
import matplotlib
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['text.latex.unicode'] = True
import matplotlib.gridspec as gridspec

sys.path.append('../../')
from tractionforce.elasticity import *
import gc
from tractionforce.norms import *
from joblib import Parallel, delayed
import multiprocessing
import pickle
import gzip

def latex_float(f):
    float_str = "{0:.2g}".format(f)
    if "e" in float_str:
        base, exponent = float_str.split("e")
        return r"{0} \times 10^{{{1}}}".format(base, int(exponent))
    else:
        return float_str

Populating the interactive namespace from numpy and matplotlib


In [4]:
padresults_1 = pickle.load(gzip.open('results/pads_2.gz', 'rb'))
padresults_2 = pickle.load(gzip.open('results/pads_4.gz', 'rb'))
padresults_3 = pickle.load(gzip.open('results/pads_8.gz', 'rb'))
padresults_4 = pickle.load(gzip.open('results/pads_16.gz','rb'))
annulusresults_1 = pickle.load(gzip.open('results/annulus_2.gz','rb'))
annulusresults_2 = pickle.load(gzip.open('results/annulus_4.gz','rb'))
annulusresults_3 = pickle.load(gzip.open('results/annulus_8.gz','rb'))
annulusresults_4 = pickle.load(gzip.open('results/annulus_16.gz','rb'))

results = [padresults_1,padresults_2,padresults_3,padresults_4,
           annulusresults_1, annulusresults_2,annulusresults_3,annulusresults_4]

In [6]:
padresults_1[-1][0]

array([  0,   2,   4,   6,   8,  10,  12,  14,  16,  18,  20,  22,  24,
        26,  28,  30,  32,  34,  36,  38,  40,  42,  44,  46,  48,  50,
        52,  54,  56,  58,  60,  62,  64,  66,  68,  70,  72,  74,  76,
        78,  80,  82,  84,  86,  88,  90,  92,  94,  96,  98, 100, 102,
       104, 106, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126, 128,
       130, 132, 134, 136, 138, 140, 142, 144, 146, 148, 150, 152, 154,
       156, 158])